In [1]:
# Import packages and functions
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [2]:
# Create an engine to connect to the SQLite database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
# Retrieve the table names from the database
insp = inspect(engine)
table_names = insp.get_table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [4]:
# Read the results of the SQL query from the Booking table into a Pandas DataFrame
df_bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)

print('Bookings table: ')
df_bookings.head()

Bookings table: 


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [5]:
# Read the results of the SQL query from the Facilities table into a Pandas DataFrame
df_facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)

print('Facilities table: ')
df_facilities.head()

Facilities table: 


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [6]:
# Read the results of the SQL query from the Members table into a Pandas DataFrame
df_members = pd.read_sql_query("SELECT * FROM Members", engine)

print('Members table: ')
df_members.head()

Members table: 


,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [7]:
# SQL query to retrieve facilities with total revenue less than 1000
query1 = """
    SELECT f.name AS facility_name,
    SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS total_revenue
    FROM Facilities AS f
    INNER JOIN Bookings AS b 
    ON f.facid = b.facid
    GROUP BY f.name
    HAVING total_revenue < 1000
    ORDER BY total_revenue;
"""

# Execute the SQL query and load the result into a Pandas DataFrame
list_of_facilities = pd.read_sql_query(query1, engine)

# Display the resulting list of facilities
print(list_of_facilities)

   facility_name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [8]:
# SQL query to retrieve members and their recommenders
query2 = """
    SELECT m.surname || ', ' || m.firstname AS members, r.surname || ', ' || r.firstname AS recommended_by
    FROM Members AS m
    INNER JOIN Members AS r 
    ON m.recommendedby = r.memid
    ORDER BY m.surname, m.firstname;
"""

# Execute the SQL query and load the result into a Pandas DataFrame
report_of_members = pd.read_sql_query(query2, engine)

# Display the resulting report
print(report_of_members)

                     members      recommended_by
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

/* Q12: Find the facilities with their usage by member, but not guests */

In [9]:
# SQL query to find facilities with usage by members, excluding guests
query3 = """
SELECT f.name AS facility_name, COUNT(b.bookid) AS member_usage
FROM Facilities AS f
INNER JOIN Bookings AS b 
ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY f.name
ORDER BY f.name;
"""

# Execute the SQL query and load the result into a Pandas DataFrame
facilities_member_usage = pd.read_sql_query(query3, engine)

# Display the resulting facilities' member usage
print(facilities_member_usage)

     facility_name  member_usage
0  Badminton Court           344
1   Massage Room 1           421
2   Massage Room 2            27
3       Pool Table           783
4    Snooker Table           421
5     Squash Court           195
6     Table Tennis           385
7   Tennis Court 1           308
8   Tennis Court 2           276


/* Q13: Find the facilities usage by month, but not guests */

In [10]:
# SQL query to find facilities' usage by month, excluding guests
query4 = """
    SELECT f.name AS facility_name, strftime('%m', b.starttime) AS month, COUNT(b.bookid) AS member_usage
    FROM Facilities AS f
    INNER JOIN Bookings AS b 
    ON f.facid = b.facid
    WHERE b.memid != 0
    GROUP BY f.name, month
    ORDER BY f.name, month;
"""
# Execute the SQL query and load the result into a Pandas DataFrame
facilities_monthly_usage = pd.read_sql_query(query4, engine)

# Display the resulting facilities' monthly usage
print(facilities_monthly_usage)

      facility_name month  member_usage
0   Badminton Court    07            51
1   Badminton Court    08           132
2   Badminton Court    09           161
3    Massage Room 1    07            77
4    Massage Room 1    08           153
5    Massage Room 1    09           191
6    Massage Room 2    07             4
7    Massage Room 2    08             9
8    Massage Room 2    09            14
9        Pool Table    07           103
10       Pool Table    08           272
11       Pool Table    09           408
12    Snooker Table    07            68
13    Snooker Table    08           154
14    Snooker Table    09           199
15     Squash Court    07            23
16     Squash Court    08            85
17     Squash Court    09            87
18     Table Tennis    07            48
19     Table Tennis    08           143
20     Table Tennis    09           194
21   Tennis Court 1    07            65
22   Tennis Court 1    08           111
23   Tennis Court 1    09           132
